In [ ]:
'''
Essa primeira versão contem a checagem de conformidade separada por perspectiva e utiliza DECLARE apenas na perspectiva de processo
As verificações de acesso e recurso são feitas "manualmente"
Pendência: filtrar atividades não reconhecidas no modelo
'''

In [63]:
import pandas as pd

logAcesso = pd.read_csv('./LogSinteticoAcessoNovo2.csv', sep=';')
logRecursos = pd.read_csv('./ModeloRecursosNovo2.csv', sep=';')
modeloAcesso = pd.read_csv('./ModeloAcesso.csv', sep=';')
logAcesso.drop('Unnamed: 0', axis=1, inplace=True)

In [1]:
import pm4py
import pandas as pd

log = pd.read_csv('./LogSinteticoAcessoOFICIAL.csv', sep=';')
event_log = pm4py.format_dataframe(log, case_id="case:concept:name",
                                   activity_key="concept:tool",
                                   timestamp_key="time:timestamp")

# Exportando para XES
pm4py.write_xes(event_log, "LogSinteticoAcessoOFICIAL.xes")

c:\Users\bruni\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 2000/2000 [00:09<00:00, 221.18it/s]


In [65]:
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog

declare_model = DeclareModel().parse_from_file('./Modelo_Log_Sintetico_Novo.decl')
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log('./LogSinteticoProcessoNovo2.xes')

parsing log, completed traces :: 100%|██████████| 2000/2000 [00:03<00:00, 539.57it/s]


In [66]:
import pm4py

log = pm4py.read_xes("./LogSinteticoProcessoNovo2.xes")

# Converter o log para DataFrame
logProcessoCSV = pm4py.convert_to_dataframe(log)
logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
logProcessoCSV

parsing log, completed traces :: 100%|██████████| 2000/2000 [00:02<00:00, 791.68it/s]


,Unnamed: 0,concept:name,lifecycle:transition,time:timestamp,concept:resource,concept:instance,@@index,@@case_index,case:concept:name
0,0,Atualizacao de requisitos funcionais,begin,2026-01-06 02:10:18.751000+00:00,re7,1,0,0,Demanda 0001
15,1,Atualizacao de requisitos funcionais,complete,2026-01-06 14:10:18.751000+00:00,re7,1,15,0,Demanda 0001
7,2,Documentacao de requisitos funcionais,begin,2026-01-06 10:10:25.751000+00:00,re6,2,7,0,Demanda 0001
16,3,Documentacao de requisitos funcionais,complete,2026-01-06 14:10:25.751000+00:00,re6,2,16,0,Demanda 0001
3,4,Contagem de ponto de funcao,begin,2026-01-06 06:26:06.751000+00:00,re6,3,3,0,Demanda 0001
...,...,...,...,...,...,...,...,...,...
79967,79967,Contagem de ponto de funcao,complete,2025-02-07 13:32:47.751000+00:00,re9,18,79967,1999,Demanda 2000
79969,79969,Executar teste,complete,2025-02-08 02:58:32.751000+00:00,re8,19,79969,1999,Demanda 2000
79971,79968,Executar teste,begin,2025-02-08 13:58:32.751000+00:00,re8,19,79971,1999,Demanda 2000
79933,79970,Atualizacao de requisitos funcionais,begin,2025-02-06 22:19:51.751000+00:00,re7,20,79933,1999,Demanda 2000


In [67]:
model_constraints = declare_model.get_decl_model_constraints()

print("Model constraints:")
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)

Model constraints:
-----------------
0 Existence1[Contagem de ponto de funcao] | |
1 Existence1[Gestao da demanda] | |
2 Existence1[Planejamento de entrega] | |
3 Existence1[Review da entrega] | |
4 Existence1[Publicar solucao] | |
5 Response[Atualizacao de requisitos funcionais, Manutencao de funcionalidade] | | |
6 Response[Documentacao de requisitos funcionais, Construcao de funcionalidade] | | |
7 Response[Planejamento de entrega, Review da entrega] | | |
8 Response[Construcao de funcionalidade, Publicar solucao] | | |
9 Response[Manutencao de funcionalidade, Publicar solucao] | | |
10 Existence1[Elaborar caso de teste] | |
11 Existence1[Executar teste] | |
12 Response[Elaborar caso de teste, Executar teste] | | |
13 Response[Documentacao de requisitos funcionais, Elaborar caso de teste] | | |
14 Response[Atualizacao de requisitos funcionais, Elaborar caso de teste] | | |
15 Precedence[Corrigir erro, Executar teste] | | |
16 Precedence[Documentacao de requisitos funcionais, Elabora

In [2]:
import pandas as pd
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import pm4py

def PreProcessData(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH):
    logAcesso = D4PyEventLog(case_name="case:concept:name")
    logAcesso.parse_xes_log(logAcessoPATH)
    logAcessoProcessado = pm4py.convert_to_dataframe(logAcesso.get_log())
    logAcessoProcessado = logAcessoProcessado.sort_values(['case:concept:name', 'concept:instance'])
    modeloRecursoProcessado = pd.read_csv(modeloRecursoPATH, sep=';')
    modeloAcessoProcessado = pd.read_csv(modeloAcessoPATH, sep=';')
    declare_model = DeclareModel().parse_from_file(modeloProcessoPATH)
    event_log = D4PyEventLog(case_name="case:concept:name")
    event_log.parse_xes_log(logProcessoPATH)
    return event_log, logAcessoProcessado, modeloRecursoProcessado, declare_model, modeloAcessoProcessado
    

In [3]:
def formatViolations(df_violations):
    '''
    Formata as violações encontradas na verificação de conformidade do Declare4Py
    '''
    violacoes = []
    for index, row in df_violations.iterrows():
        for coluna in df_violations.columns:
            if row[coluna] == 1:
                violacoes.append(f"Trace {index} violou {coluna}")
    return violacoes

In [4]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser

def CheckProcessConformance(logProcesso, modeloProcesso):
    '''
    Checa conformidade de processo entre o log e o modelo DECLARE com a biblioteca Declare4Py
    Recebe um log de processo (em formato EventLog) e um modelo de processo (em DeclareModel)
    
    '''
    model_constraints = modeloProcesso.get_decl_model_constraints()

    # print("Model constraints:")
    # print("-----------------")
    # for idx, constr in enumerate(model_constraints):
    #     print(idx, constr)
    basic_checker = MPDeclareAnalyzer(log=logProcesso, declare_model=modeloProcesso, consider_vacuity=False)
    conf_check_res: MPDeclareResultsBrowser = basic_checker.run()
    violacoes = formatViolations(conf_check_res.get_metric(metric="num_violations"))
    return violacoes

In [5]:
def manter_maiusculas(valor):
    if isinstance(valor, str):
        return ''.join([c for c in valor if c.isupper()])
    return ''

def CheckAcessConformance(logProcesso,logAcesso, modeloAcesso):
    '''
    Checa conformidade entre o log de acesso, log de processo e o modelo de acesso
    Recebe modelo de acesso e logs em csv
    '''
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    violacoes = {}
    violacoes['OperacaoProibida'] = []
    violacoes['AcessoObgNaoRealizado'] = []
    logProcessoFiltrado = logProcessoCSV[logProcessoCSV["lifecycle:transition"] == "begin"]
    logAcessoObg = modeloAcesso.copy()
    logAcessoObg.loc[:, logAcessoObg.columns != "Ferramenta"] = logAcessoObg.loc[:, logAcessoObg.columns != "Ferramenta"].map(manter_maiusculas)
    for index, row in logProcessoFiltrado.iterrows():
        demanda = row['case:concept:name']
        atividade = row['concept:instance']
        nome = row['concept:name']
        acessos = logAcesso[(logAcesso['case:concept:name'] == demanda) & (logAcesso['concept:instance'] == atividade)]
        modeloAtv = modeloAcesso[['Ferramenta', nome]]
        modeloAtvObg = logAcessoObg[['Ferramenta', nome]].copy()
        modeloAtvObg.loc[:,'Realizada'] = modeloAtvObg[nome].apply(lambda x: False if (isinstance(x, str) and len(x) > 0) else True)
        for i, acc in acessos.iterrows():
            ferramenta = acc['concept:tool']
            opPermitidas = modeloAtv[modeloAtv["Ferramenta"] == ferramenta][nome].iloc[0].split(', ') if isinstance(modeloAtv[modeloAtv["Ferramenta"] == ferramenta][nome].iloc[0], str) else []
            if acc['concept:operation'] not in opPermitidas:
                violacoes['OperacaoProibida'].append([row["concept:name"], demanda, acc['concept:operation'], ferramenta])
            if acc['concept:operation'].lower() == modeloAtvObg[modeloAtvObg["Ferramenta"] == ferramenta][nome].iloc[0].lower():
                modeloAtvObg.loc[modeloAtvObg["Ferramenta"] == ferramenta, "Realizada"] = True
        if not modeloAtvObg['Realizada'].all():
            naoRealizados = modeloAtvObg.loc[modeloAtvObg['Realizada'] == False, 'Ferramenta']     
            violacoes['AcessoObgNaoRealizado'].append([row["concept:name"], demanda, ', '.join(naoRealizados.tolist())])
    return violacoes
    

In [6]:
import pm4py

def CheckResourceConformance(logProcesso, logAcesso, modeloRecurso):
    '''
    Checa conformidade entre o log de acesso, log de processo e o modelo de recurso
    Recebe modelo de recurso e logs em csv
    '''
    
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    logProcessoFiltrado = logProcessoCSV[logProcessoCSV["lifecycle:transition"] == "begin"]
    
    violacoes = {}
    violacoes["AcessoRecursoEquipe"] = []
    violacoes["AcessoRecursoErrado"] = []
    violacoes["Atividade"] = []
    for index, row in modeloRecurso.iterrows():
        demanda = row['case:concept:name']
        recursos = row['concept:resources'].split(", ")
        atividades = logProcessoFiltrado[logProcessoFiltrado['case:concept:name'] == demanda]
        acessos = logAcesso[logAcesso['case:concept:name'] == demanda]
        for i, atv in atividades.iterrows():
            recursoAtv = atv['concept:resource']
            if recursoAtv not in recursos:
                violacoes["Atividade"].append([atv['concept:name'], demanda, recursoAtv])
            
            acessosAtv = acessos[acessos['concept:instance']== atv['concept:instance']]
            for j, acc in acessosAtv.iterrows():
                if recursoAtv != acc['concept:resource']:
                    violacoes["AcessoRecursoErrado"].append([acc['concept:tool'], demanda, acc['concept:resource'], recursoAtv])
                if acc['concept:resource'] not in recursos:
                    violacoes["AcessoRecursoEquipe"].append([acc['concept:tool'], demanda, acc['concept:resource']])
    return violacoes

In [7]:
def formatInconformances(conformanceProcess, conformanceAcess, conformanceResource):
    '''
    Formata as inconformidades encontradas em cada verificação (pode vir a combinar os padrões de anomalias no futuro)
    '''
    print('Violações de fluxo de processo:')
    for violation in conformanceProcess:
        print(violation)
    
    print("Violações de Acesso:")
    for key, violation in conformanceAcess.items():
        if key == "OperacaoProibida":
            for occur in violation:
                print(f'Violação de Acesso na atividade {occur[0]} no trace {occur[1]}, a operação {occur[2]} não era permitida para a ferramenta {occur[3]}')
        if key == "AcessoObgNaoRealizado":
            for occur in violation:
                print(f"Violação de Acesso na atividade {occur[0]} no trace {occur[1]}, os acessos obrigatórios às ferramentas {occur[2]} não foram realizados")
    
    
    print('Violações de Privacidade:')
    for key, violation in conformanceResource.items():
        if key == "Atividade":
            for occur in violation:
                print(f'Violação de Privacidade na atividade {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não faz parte da equipe determinada para realizar a demanda')
        if key == "AcessoRecursoEquipe":
            for occur in violation:
                print(f'Violação de Privacidade no acesso a {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não faz parte da equipe determinada para realizar a demanda')
        if key == "AcessoRecursoErrado":
            for occur in violation:
                print(f'Violação de Privacidade no acesso a {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não era o designado da atividade vinculada e sim o {occur[3]}')
    

In [8]:
def MultiperspectiveConformanceAlgorithm(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH):
  '''
  O algoritmo recebe: um log de processo, um log de acesso a dados, um modelo de recurso, um modelo DECLARE de processo e um modelo de acesso a dados
  '''
  logProcesso, logAcesso, modeloRecurso, modeloProcesso, modeloAcesso = PreProcessData(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH)
  conformanceProcess = CheckProcessConformance(logProcesso, modeloProcesso)
  conformanceAcess = CheckAcessConformance(logProcesso, logAcesso, modeloAcesso)
  conformanceResource = CheckResourceConformance(logProcesso, logAcesso, modeloRecurso)
  return formatInconformances(conformanceProcess, conformanceAcess, conformanceResource)

In [10]:
MultiperspectiveConformanceAlgorithm('./LogSinteticoProcessoOFICIAL.xes', './LogSinteticoAcessoOFICIAL.xes', './ModeloRecursosOFICIAL.csv', './Modelo_Log_Sintetico_OFICIAL.decl', './ModeloAcessoOFICIAL.csv')

parsing log, completed traces :: 100%|██████████| 2000/2000 [00:06<00:00, 321.88it/s]


Violações de fluxo de processo:
Violações de Acesso:
Violação de Acesso na atividade Atualizacao de requisitos funcionais no trace Demanda 0001, a operação r não era permitida para a ferramenta PF
Violação de Acesso na atividade Atualizacao de requisitos funcionais no trace Demanda 0001, os acessos obrigatórios às ferramentas Requisito não foram realizados
Violações de Privacidade:
Violação de Privacidade no acesso a Gestao no trace Demanda 0001, o recurso re11 não era o designado da atividade vinculada e sim o re1
Violação de Privacidade no acesso a Requisito no trace Demanda 0001, o recurso re11 não era o designado da atividade vinculada e sim o re1
Violação de Privacidade no acesso a Requisito no trace Demanda 0001, o recurso re11 não era o designado da atividade vinculada e sim o re1
Violação de Privacidade no acesso a Requisito no trace Demanda 0001, o recurso re11 não era o designado da atividade vinculada e sim o re1
Violação de Privacidade no acesso a Codigo no trace Demanda 00